In [5]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
import logging
import logging.config
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder
from sklearn.model_selection import train_test_split

# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,hamming_loss

import os
import commonfunction

In [6]:
logging.basicConfig()
logging.config.fileConfig(os.path.abspath('../config/logging.conf'))
logger = logging.getLogger(__name__)
logger.debug('Testing ******  ==============')
logger.debug("Retesting the same **** ===========")

logger.debug('-------------- Testing  ============== ')
logger.debug("-------------- Retesting the same  ==============")

2018-07-20 15:23:06,777 - __main__ - DEBUG - Testing ******  ==============
2018-07-20 15:23:06,777 - __main__ - DEBUG - Retesting the same **** ===========
2018-07-20 15:23:06,778 - __main__ - DEBUG - -------------- Testing  ============== 
2018-07-20 15:23:06,778 - __main__ - DEBUG - -------------- Retesting the same  ==============


In [25]:
train_df = pd.read_csv("../../data/processed/train.csv")
test_df = pd.read_csv("../../data/processed/test.csv")

In [26]:
le  = LabelBinarizer()
cat_arr = le.fit_transform(train_df['cat'])

cat_df = pd.DataFrame(data=cat_arr,columns=le.classes_)

train_df[pd.isnull(train_df['subcat'])]  = 'NaN'
subcat_arr = le.fit_transform(train_df['subcat'])

subcat_df = pd.DataFrame(data=subcat_arr,columns=le.classes_)

Y = pd.concat([cat_df,subcat_df],axis=1)

In [27]:
Y.drop('NaN',inplace=True,axis=1)

In [28]:
col_todrop = ['cat','subcat','Unnamed: 0','Unnamed: 0.1','document_id','no_pages','no_words','ques_cnt']
for col in col_todrop:
    train_df.drop(col,axis=1,inplace=True)  
    
col_todrop = ['Unnamed: 0','Unnamed: 0.1','document_id','no_pages','no_words','ques_cnt']
for col in col_todrop:
    test_df.drop(col,axis=1,inplace=True)

X_train, X_test, Y_train, Y_test = train_test_split(train_df['content'], Y, test_size=0.33, random_state=42)

In [29]:
from skmultilearn.problem_transform import LabelPowerset
pipe = Pipeline([
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('BR_GB', LabelPowerset(GaussianNB())),
])

In [30]:
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
#classifier = BinaryRelevance(GaussianNB())

# train
pipe.fit(X_train, Y_train)

# predict
prediction = pipe.predict(X_test)

print('Test accuracy is {}'.format(accuracy_score(Y_test, prediction)))

Test accuracy is 0.634429400387


In [31]:
print('Hamming loss is {}'.format(hamming_loss(Y_test, prediction)))

Hamming loss is 0.0549875656259
